You now know the following

1. Generate open-loop control from a given route

2. Simulate vehicular robot motion using bicycle/ unicycle model

Imagine you want to make an utility for your co-workers to try and understand vehicle models. 
Dashboards are common way to do this.

There are several options out there : Streamlit, Voila, Observable etc

Follow this
<a href="https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e">Medium post</a> on Jupyter Dash and see how to package what you learnt today in an interactive manner

Here is a   <a href="https://stackoverflow.com/questions/53622518/launch-a-dash-app-in-a-google-colab-notebook">stackoverflow question </a> on how to run dash applications on Collab

What can you assume?
+ Fix $v,\omega$ or $v,\delta$ depending on the model (users can still pick the actual value)
+ fixed wheelbase for bicycle model

Users can choose 
+ unicycle and bicycle models
+ A pre-configured route ("S", "inverted-S", "figure-of-eight" etc)
+ 1 of 3 values for $v, \omega$ (or $\delta$) 

In [2]:
pip install jupyter-dash -q

     |████████████████████████████████| 81kB 4.3MB/s 
     |████████████████████████████████| 1.0MB 32.3MB/s 
     |████████████████████████████████| 3.5MB 39.0MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 1.8MB 45.0MB/s 
     |████████████████████████████████| 358kB 34.7MB/s 


In [18]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Load Data
# df = px.data.tips()
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
                       html.H1("Vehicle Simulation models"),
                       dcc.Graph(id='graph'),
                       html.Br(),
                       html.Label([
                                   "model",
                                   dcc.RadioItems(
                                       id='model-options', 
                                       value='unicycle',options=[{'label':'unicycle model','value':'unicycle'},{'label':'bicycle model','value':'bicycle'}]
                                   )
                       ]),
                       html.Br(),
                       html.Label([
                                   "route",
                                   dcc.RadioItems(
                                       id='route-options', 
                                       value='s',options=[
                                                          {'label':'S','value':'s'},
                                                          {'label':'inverted-S','value':'smirror'},
                                                          {'label':'figure-of-eight','value':'eight'}]
                                   )           
                       ]),
                       html.Br(),
                       html.Label([
                                   "straight_line_velocity",
                                   dcc.Slider(
                                       id='velocity-value-straight', value = 1,
                                       min=0,max=5,step=0.5,
                                       marks={i: '{}'.format(i) for i in np.arange(0,5,0.5)}
                                   )
                       ]),
                       html.Br(),
                       html.Label([
                                   "turn_velocity",
                                   dcc.Slider(
                                       id='velocity-value-turn', value = 0.5,
                                       min=0,max=3,step=0.2,
                                       marks={i: '{}'.format(i) for i in np.arange(0,3,0.2)}
                                   )
                       ]),
                       html.Br(),
                       html.Label([
                                   "step_angular_velocity",
                                   dcc.Slider(
                                       id='angular-velocity-value', value = 0.2,
                                       min=0,max=2,step=0.1,
                                       marks={i: '{}'.format(i) for i in np.arange(0,2,0.1)}
                                   )
                       ])
                      #  html.Div(id='graph')
])
# , style={'columnCount': 2}

# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    Input('model-options', 'value'),
    Input('route-options', 'value'),
    Input('velocity-value-straight', 'value'),
    Input('velocity-value-turn', 'value'),
    Input('angular-velocity-value', 'value')
)
def update_figure(model_choice,route_choice,vc_fast,vc_slow,ang_vel):
    dt = 0.1
    pose = (0,0,np.pi/2)
    st_dist = 10
    routes = {'s':[("right",90), ("straight", st_dist), ("left", 90), ("straight", st_dist), ("left", 90), ("straight", st_dist), ("right", 90),("straight", st_dist), ("right", 90), ("straight", st_dist)],
              'smirror':[("left",90), ("straight", st_dist), ("right", 90), ("straight", st_dist), ("right", 90), ("straight", st_dist), ("left", 90),("straight", st_dist), ("left", 90), ("straight", st_dist)],
              'eight':[("right",90), ("straight", st_dist), ("left", 90), ("straight", st_dist), ("left", 90), ("straight", st_dist), ("right", 90),("straight", st_dist), ("right", 90), ("straight", st_dist),
                       ("right", 90), ("straight", st_dist), ("right", 90), ("straight", st_dist), ("left", 90),("straight", st_dist), ("left", 90), ("straight", st_dist)]}
    route = routes[route_choice]
    vcfast = vc_fast
    vcslow = vc_slow
    wc = ang_vel
    if model_choice == 'unicycle':      
        robot_trajectory = unicycle_model(pose, route, vcfast,vcslow, wc,dt)
    elif model_choice == 'bicycle':
        robot_trajectory = bicycle_model(pose, route, vcfast,vcslow, wc,dt)
    fig = px.scatter(x=robot_trajectory[:,0], y=robot_trajectory[:,1])
    fig.layout.yaxis.scaleanchor = 'x'
    fig.layout.height = 600
    fig.layout.width = 600
    fig.layout.title = 'vehicle path'
    fig.update_traces(mode='lines+markers')
    return fig

def unicycle_model(pose, route,vcfast,vcslow, wc, dt = 0.01):
    all_w = []
    all_v = []
    deltas = {"straight":0, "left":wc, "right":-wc}
    for maneuver,value in route:
        u = deltas[maneuver]
        angle_steps = np.ceil(np.deg2rad(value)/(wc*dt))
        all_w += [u]*np.ceil(value/(vcfast*dt)).astype(int) if maneuver == "straight" else [u]*angle_steps.astype(int)
        all_v += [vcfast]*np.ceil(value/(vcfast*dt)).astype(int) if maneuver == "straight" else [vcslow]*angle_steps.astype(int)
    robot_trajectory = []
    for v, w in zip(all_v, all_w):
        x, y, theta = pose
        x += v*np.cos(theta)*dt
        y += v*np.sin(theta)*dt
        theta += w*dt
        theta = np.arctan2(np.sin(theta), np.cos(theta))
        new_pose = (x, y, theta)
        robot_trajectory.append(pose)
        pose = new_pose 
    robot_trajectory = np.array(robot_trajectory)
    return robot_trajectory

def bicycle_model(pose, route,vcfast,vcslow, deltac, dt = 0.01):
    all_delta = []
    all_v = []
    deltas = {"straight":0, "left":deltac, "right":-deltac}
    for maneuver,angle in route:
        u = deltas[maneuver]
        angle_steps = np.ceil(np.deg2rad(angle)*0.9/(vcslow*np.tan(deltac)*dt))
        all_delta += [u]*np.ceil(angle/(vcfast*dt)).astype(int) if maneuver == "straight" else [u]*angle_steps.astype(int)
        all_v += [vcfast]*np.ceil(angle/(vcfast*dt)).astype(int) if maneuver == "straight" else [vcslow]*angle_steps.astype(int)
    robot_trajectory = []
    for v, delta in zip(all_v, all_delta):
        x, y, theta = pose
        x += v*np.cos(theta)*dt
        y += v*np.sin(theta)*dt
        theta += (v/0.9)*np.tan(delta)*dt
        theta = np.arctan2(np.sin(theta), np.cos(theta))
        new_pose = (x, y, theta)
        robot_trajectory.append(pose) 
        pose = new_pose
    robot_trajectory = np.array(robot_trajectory)
    return robot_trajectory

# Run app and display result inline in the notebook
app.run_server(mode='inline')

<IPython.core.display.Javascript object>